In [ ]:
from edf_interface import data
from edf_interface.data.dataset import load_demos
from edf_interface.data import preprocess

from torch_cluster import fps

In [ ]:
# episodes = load_demos('demo/panda_mug_on_hanger/')
# episodes = load_demos('demo/panda_bowl_on_dish/')
episodes = load_demos('demo/panda_bottle_on_shelf/')

In [ ]:
episode_idx = 3
episode: data.DemoSequence = episodes[episode_idx]
pick_demo: data.TargetPoseDemo = episode[0]
try:
    place_demo: data.TargetPoseDemo = episode[1]
except IndexError:
    place_demo = None

In [ ]:
viz_pcd = pick_demo.scene_pcd
viz_pcd = preprocess.crop_bbox(viz_pcd, bbox=[[-0.4, 0.4], [-0.8, 0.8], [0.5, 1.3]], targets=['scene_pcd'])
viz_pcd = preprocess.downsample(viz_pcd, voxel_size=0.01, coord_reduction='average')

In [ ]:
multiscale_pcds = [viz_pcd]
pool_ratios = [0.15, 0.3, 0.3, 0.3]
for pool_ratio in pool_ratios:
    curr_pcd = multiscale_pcds[-1]
    node_dst_idx = fps(src=curr_pcd.points, ratio=pool_ratio)
    multiscale_pcds.append(
        curr_pcd.new(
            points=curr_pcd.points.detach().index_select(index=node_dst_idx, dim=0).clone(),
            colors=curr_pcd.colors.detach().index_select(index=node_dst_idx, dim=0).clone()
        )
    )

In [ ]:
multiscale_pcds[0].show(point_size=2.)

In [ ]:
multiscale_pcds[1].show(point_size=3.)

In [ ]:
multiscale_pcds[2].show(point_size=5.)

In [ ]:
multiscale_pcds[3].show(point_size=5.)

In [ ]:
multiscale_pcds[4].show(point_size=5.)